In [69]:
import json
lines = open('../data/feature_map.json').readlines()
fm = json.loads(lines[0])
fm_inverse = {v: k for k, v in fm.iteritems()}
#fuser = json.loads(lines[1])
#fsku = json.loads(lines[2])

In [93]:
from scipy.sparse import csr_matrix

def construct(fpath):
    row, col, val = [], [], []
    user_id, sku_id = [], []
    y = []
    
    with open(fpath) as f:
        for i, line in enumerate(f):
            obj = json.loads(line)
            user_id.append(str(int(float(obj['user_id']))))
            sku_id.append(str(obj['sku_id']))
            #if user_id[-1] in fuser:
            #    for k, v in fuser[user_id[-1]].iteritems():
            #        row.append(i)
            #        col.append(k)
            #        val.append(v)
            #if sku_id[-1] in fsku:
            #    for k, v in fsku[sku_id[-1]].iteritems():
            #        row.append(i)
            #        col.append(k)
            #        val.append(v)
            y.append(obj['label'])
            for j, o in enumerate(obj['interval_actions']):
                for k, v in o.iteritems():
                    if k == '6':
                        row.append(i)
                        col.append(fm['action_%s_%s' % (j, k)])
                        val.append(v)
    return user_id, sku_id, y, csr_matrix((val, (row, col)), shape=(len(y), len(fm)))

In [94]:
X_user_id, X_sku_id, X_y, X = construct('../data/action_04_09.json')
T_user_id, T_sku_id, T_y, T = construct('../data/action_11_16.json')
#Z_user_id, Z_sku_id, zzzzz, Z = construct('../data/action_16_21.json')

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, X_y, test_size=0.33, random_state=2333)

In [86]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
model = XGBClassifier()
model_X = XGBClassifier()
model_T= XGBClassifier()
#model = LogisticRegression()

In [87]:
model.fit(X_train, y_train)
from sklearn import metrics
pred_train = model.predict_proba(X_train)
pred_test = model.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_train, pred_train[:,1])
print(metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_test[:,1])
print(metrics.auc(fpr, tpr))

0.82396222886
0.798319949224


In [95]:
model_T.fit(T, T_y)
model_X.fit(X, X_y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [89]:
def top_features(m, n = 20):
    fs = [(fm_inverse[i], v) for i, v in enumerate(m.feature_importances_)]
    fs.sort(key=lambda o: o[1], reverse=True)
    return fs[0: n]

In [96]:
top_features(model_T, n = 20)

[(u'action_2_6', 0.21354933),
 (u'action_7_6', 0.16936672),
 (u'action_8_6', 0.15022092),
 (u'action_4_6', 0.12371134),
 (u'action_9_6', 0.1089838),
 (u'action_1_6', 0.075110458),
 (u'action_5_6', 0.073637702),
 (u'action_6_6', 0.03976436),
 (u'action_0_6', 0.029455081),
 (u'action_3_6', 0.016200295),
 (u'action_9_5', 0.0),
 (u'action_9_4', 0.0),
 (u'action_9_1', 0.0),
 (u'action_9_3', 0.0),
 (u'action_9_2', 0.0),
 (u'action_8_4', 0.0),
 (u'action_8_5', 0.0),
 (u'action_8_1', 0.0),
 (u'action_8_2', 0.0),
 (u'action_8_3', 0.0)]

In [97]:
top_features(model_X, n = 20)

[(u'action_7_6', 0.20930232),
 (u'action_2_6', 0.20494185),
 (u'action_4_6', 0.16715117),
 (u'action_8_6', 0.1380814),
 (u'action_5_6', 0.08139535),
 (u'action_3_6', 0.072674416),
 (u'action_9_6', 0.049418606),
 (u'action_1_6', 0.042151164),
 (u'action_6_6', 0.024709303),
 (u'action_0_6', 0.010174419),
 (u'action_9_5', 0.0),
 (u'action_9_4', 0.0),
 (u'action_9_1', 0.0),
 (u'action_9_3', 0.0),
 (u'action_9_2', 0.0),
 (u'action_8_4', 0.0),
 (u'action_8_5', 0.0),
 (u'action_8_1', 0.0),
 (u'action_8_2', 0.0),
 (u'action_8_3', 0.0)]

In [92]:
top_features(model, n = 20)

[(u'action_2_6', 0.18168168),
 (u'action_4_6', 0.17717718),
 (u'action_8_6', 0.13813815),
 (u'action_7_6', 0.13363363),
 (u'action_5_6', 0.1021021),
 (u'action_3_6', 0.1021021),
 (u'action_9_6', 0.061561562),
 (u'action_0_6', 0.061561562),
 (u'action_1_6', 0.025525525),
 (u'action_6_6', 0.016516516),
 (u'action_9_5', 0.0),
 (u'action_9_4', 0.0),
 (u'action_9_1', 0.0),
 (u'action_9_3', 0.0),
 (u'action_9_2', 0.0),
 (u'action_8_4', 0.0),
 (u'action_8_5', 0.0),
 (u'action_8_1', 0.0),
 (u'action_8_2', 0.0),
 (u'action_8_3', 0.0)]

In [98]:
X_pred = model_X.predict_proba(X)
calculate_f(get_result(X_user_id, X_sku_id, X_pred, 0.02), build_target(X_user_id, X_sku_id, X_y))

0.07919561881547288

In [99]:
T_pred = model_X.predict_proba(T)
calculate_f(get_result(T_user_id, T_sku_id, T_pred, 0.02), build_target(T_user_id, T_sku_id, T_y))

0.03802271396195666

In [21]:
from collections import defaultdict

def build_target(user_id, sku_id, y):
    target = defaultdict(set)
    for user, sku, l in zip(user_id, sku_id, y):
        if l == 1:
            target[user].add(sku)
    return target

In [22]:
def get_result(user_id, sku_id, pred, threshold):
    src = {}
    for user, sku, l in zip(user_id, sku_id, pred[:, 1]):
        if l >= threshold:
            if user in src:
                if l > src[user][1]:
                    src[user] = (sku, l)
            else:
                src[user] = (sku, l)
    return src

In [23]:
def calculate_f(src, target):
    f1p = len(set(src.keys()).intersection(set(target.keys()))) * 1.0 / len(src.keys()) 
    f1r = len(set(src.keys()).intersection(set(target.keys()))) * 1.0 / len(target.keys())
    
    c = 0
    for k, v in src.items():
        if v[0] in target.get(k, set()):
            c += 1
    f2p = c * 1.0 / len(src.keys())
    f2r = c * 1.0 / len(target.keys())
    
    f1=6*f1r*f1p/(5*f1r+f1p)
    f2=5*f2r*f2p/(2*f2r+3*f2p)
    return 0.4*f1 + 0.6*f2

In [24]:
src = get_result(Z_user_id, Z_sku_id, model.predict_proba(Z), 0.02)

In [25]:
from __future__ import print_function

with open('out.csv', 'w') as f:
    print('user_id,sku_id', file=f)
    for i, (k, v) in enumerate(src.iteritems()):
        print('%s,%s' % (int(float(k)), v[0]), file=f)